# Demo 2: Setting Up Prompt Baking with Bread AI

This notebook demonstrates how to use Bread AI to "bake" a system prompt into model weights, eliminating the need for runtime system prompts.

**Process:**
1. Load the system prompt (expertise definition)
2. Configure the baking process
3. Generate synthetic training data (stim)
4. Run rollout with prompted model
5. Bake the behavior into weights

## What is Prompt Baking?

Prompt baking encodes prompt behavior directly into model weights. After baking, the model exhibits the prompted behavior with **ZERO input tokens** required at inference time.

Think of it like:
- **Traditional**: Giving someone instructions every single time
- **Baked**: Training someone once, then they just know

## Setup

In [1]:
import os
import sys
from pathlib import Path
import yaml

sys.path.append(str(Path(".").resolve().parent))

from utils.helpers import load_system_prompt, count_tokens

## Step 1: Load Configuration

In [2]:
# Load the expert system prompt
system_prompt = load_system_prompt()
print(f"Loaded system prompt: {count_tokens(system_prompt)} tokens")

# Load bake configuration
with open("../configs/bake_config.yaml", 'r') as f:
    bake_config = yaml.safe_load(f)

print(f"Loaded bake configuration")
print(f"  Repository: {bake_config['repository']['name']}")
print(f"  Base model: {bake_config['bake']['base_model']}")
print(f"  Method: {bake_config['bake']['method']}")

Loaded system prompt: 551 tokens
Loaded bake configuration
  Repository: rag-expert-model
  Base model: gpt-4
  Method: weight-space-editing


## Mock Bread AI Client

In production, use `from bread import BreadClient`

In [3]:
class MockBreadClient:
    """Mock Bread AI client for demonstration"""
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.repo = None
    
    def create_repository(self, name: str, description: str):
        print(f"   Created repository: {name}")
        self.repo = {"name": name, "description": description}
        return self.repo
    
    def create_prompt(self, repo_name: str, prompt_text: str, metadata: dict):
        print(f"   Registered prompt: {metadata.get('name', 'unnamed')}")
        print(f"   Tokens: ~{count_tokens(prompt_text)}")
        return {"id": "prompt_123", "tokens": count_tokens(prompt_text)}
    
    def create_stim_job(self, repo_name: str, config: dict):
        print(f"   Started stim generation job")
        print(f"   Target: {config.get('num_samples', 100)} samples")
        return {"job_id": "stim_456", "status": "running"}
    
    def create_rollout_job(self, repo_name: str, stim_job_id: str, config: dict):
        print(f"   Started rollout job")
        return {"job_id": "rollout_789", "status": "running"}
    
    def create_bake_job(self, repo_name: str, rollout_job_id: str, config: dict):
        print(f"   Started baking job")
        print(f"   Base model: {config.get('base_model', 'gpt-4')}")
        print(f"   Method: {config.get('method', 'weight-space-editing')}")
        return {"job_id": "bake_012", "status": "running"}
    
    def get_baked_model(self, repo_name: str, version: str = "latest"):
        return {
            "model_id": "rag-expert-baked-v1",
            "repo": repo_name,
            "version": version,
            "status": "ready"
        }

# Initialize client
client = MockBreadClient(api_key="demo_key")
print("Bread AI client initialized")

Bread AI client initialized


## Step 2: Create Model Repository

Bread AI uses git-like version control for models.

In [4]:
repo_config = bake_config["repository"]

repo = client.create_repository(
    name=repo_config["name"],
    description=repo_config["description"]
)

   Created repository: rag-expert-model


## Step 3: Register System Prompt for Baking

In [5]:
prompt_config = bake_config["prompts"][0]

prompt = client.create_prompt(
    repo_name=repo["name"],
    prompt_text=system_prompt,
    metadata={"name": prompt_config["name"]}
)

   Registered prompt: expert_system_prompt
   Tokens: ~551


## Step 4: Generate Synthetic Training Data (Stim)

**Stim** = Diverse user queries that test the expert behavior. The model will be tested on these queries during rollout.

In [6]:
stim_config = bake_config["generators"]["stim"]

print("Stim configuration:")
print(f"  Model: {stim_config['model']}")
print(f"  Samples: {stim_config['num_samples']}")
print(f"  Topics: {', '.join(stim_config['topics'][:3])}...")

stim_job = client.create_stim_job(
    repo_name=repo["name"],
    config=stim_config
)

print(f"\nGenerating synthetic data...")
print(f"Generated {stim_config['num_samples']} diverse queries")

Stim configuration:
  Model: gpt-4
  Samples: 100
  Topics: vector database architecture questions, RAG system design problems, embedding model selection...
   Started stim generation job
   Target: 100 samples

Generating synthetic data...
Generated 100 diverse queries


## Step 5: Run Rollout (Prompted Model Responses)

**Rollout** = Run the model WITH system prompt on stim data. This captures how the expert model should behave.

In [7]:
rollout_config = bake_config["generators"]["rollout"]

print("Rollout configuration:")
print(f"  Model: {rollout_config['model']}")
print(f"  Use system prompt: {rollout_config['use_system_prompt']}")

rollout_job = client.create_rollout_job(
    repo_name=repo["name"],
    stim_job_id=stim_job["job_id"],
    config=rollout_config
)

print(f"\nCollecting expert responses...")
print(f"Collected {stim_config['num_samples']} expert responses")

Rollout configuration:
  Model: gpt-4
  Use system prompt: True
   Started rollout job

Collected 100 expert responses


## Step 6: Bake Behavior into Model Weights

**Baking** = Surgically edit model weights to internalize behavior. This uses advanced model editing techniques (not traditional fine-tuning).

In [8]:
bake_job_config = bake_config["bake"]

print("Bake configuration:")
print(f"  Base model: {bake_job_config['base_model']}")
print(f"  Method: {bake_job_config['method']}")
print(f"  Preserve general capabilities: {bake_job_config['constraints']['preserve_general_capabilities']}")

bake_job = client.create_bake_job(
    repo_name=repo["name"],
    rollout_job_id=rollout_job["job_id"],
    config=bake_job_config
)

print(f"\nApplying weight-space edits...")
print(f"Preserving general capabilities...")
print(f"Validating behavior preservation...")
print(f"Baking complete!")

Bake configuration:
  Base model: gpt-4
  Method: weight-space-editing
  Preserve general capabilities: True
   Started baking job
   Base model: gpt-4
   Method: weight-space-editing

Applying weight-space edits...
Preserving general capabilities...
Validating behavior preservation...
Baking complete!


## Step 7: Retrieve Baked Model

In [9]:
baked_model = client.get_baked_model(repo_name=repo["name"])

print(f"Baked model ready!")
print(f"  Model ID: {baked_model['model_id']}")
print(f"  Status: {baked_model['status']}")
print(f"  Version: {baked_model['version']}")

Baked model ready!
  Model ID: rag-expert-baked-v1
  Status: ready
  Version: latest


## Summary: What Just Happened

1. Loaded ~550-token expert system prompt
2. Generated 100 diverse test queries
3. Collected expert responses WITH system prompt
4. Surgically edited model weights to internalize behavior
5. Created new model with expertise "baked in"

## Key Benefits

- Expert behavior is now **IN THE WEIGHTS**
- **Zero** system prompt tokens needed at inference
- **Faster** response times (no prompt processing)
- **Consistent** behavior guaranteed
- **Version-controlled** expertise (git-like workflow)

## Next Step

Run **Demo 3** to use the baked model and compare results!